In [1]:
import pandas as pd
import numpy as np
from math import log2

In [2]:
class Node_ID3:
    
    def __init__(self, data):
        self.feature = None
        self.children = {}
        self.data = data
        self.classification = None

        if self.get_entropy(data) == 0:
            self.is_leaf = True
            self.classification = self.data['Decision'].iloc[0]
            return
        
        self.is_leaf = False
        self.set_feature()

    def split(self, category):
        return self.data.groupby(category)

    def get_info_gain(self, category):
        info_gain = self.get_entropy(self.data)
        
        for name, group in self.split(category):
            category_entropy = self.get_entropy(group)
            category_prob = len(group) / len(self.data)
            info_gain -= category_entropy * category_prob

        return info_gain

    def get_entropy(self, data):
        outcomes = np.unique(data['Decision'])
        entropy = 0
        for o in outcomes:
            e = len(data[data['Decision'] == o]) / len(data)
            entropy -= e * log2(e)

        return entropy
    
    def get_best_feature(self):
        best_info_gain = -float('inf')
        best_split_feature = None
        
        for feature in self.data:

            if feature == 'Decision': continue

            potential_info_gain = self.get_info_gain(feature)
            
            if potential_info_gain > best_info_gain:
                best_info_gain = potential_info_gain
                best_split_feature = feature

        return best_split_feature

    def set_feature(self):
        self.feature = self.get_best_feature()
        for name, group in self.split(self.feature):
            self.children[name] = Node_ID3(group)

    def __str__(self, level=0, cat=""):
        print_value = self.classification if self.is_leaf else f"Split by {self.feature}"
        
        ret = "\t" * level + cat + ': ' + print_value + "\n"
        for key in self.children:
            ret += self.children[key].__str__(level+1, key)
        return ret


In [3]:
class NodeC4_5(Node_ID3):

    def split_info(self, category):
        split_info = 0
        for name, group in self.split(category):
            category_probability = len(group) / len(self.data)
            split_info -= category_probability * log2(category_probability)

        return split_info
    
    def get_info_gain(self, category):
        info_gain = super().get_info_gain(category)
        split_info = self.split_info(category)

        return info_gain / split_info

In [4]:
class Node_CART:

    def __init__(self, data):
        self.feature = None
        self.children = {}
        self.data = data
        self.classification = None

        if self.get_individual_gini(data) == 0:
            self.is_leaf = True
            self.classification = self.data['Decision'].iloc[0]
            return
        
        self.is_leaf = False
        self.set_feature()

    def split(self, category):
        return self.data.groupby(category)

    def get_individual_gini(self, data):
        outcomes = np.unique(data['Decision'])
        gini = 1
        for o in outcomes:
            p = len(data[data['Decision'] == o]) / len(data)
            gini -= p ** 2

        return gini

    def get_gini(self, category):
        ginis = []
        for name, group in self.split(category):
            category_probability = len(group) / len(self.data)
            ginis.append(self.get_individual_gini(group) * category_probability)

        return sum(ginis) / len(ginis)

    def get_best_feature(self):
        min_gini = float('inf')
        best_split_feature = None
        
        for feature in self.data:

            if feature == 'Decision': continue

            feature_gini = self.get_gini(feature)
            
            if feature_gini < min_gini:
                min_gini = feature_gini
                best_split_feature = feature

        return best_split_feature
    
    def set_feature(self):
        self.feature = self.get_best_feature()
        for name, group in self.split(self.feature):
            self.children[name] = Node_CART(group)

    def __str__(self, level=0, cat=""):
        print_value = self.classification if self.is_leaf else f"Split by {self.feature}"
        
        ret = "\t" * level + cat + ': ' + print_value + "\n"
        for key in self.children:
            ret += self.children[key].__str__(level+1, key)
        return ret
        

In [5]:
class DecisionTree:

    def __init__(self, data_path, node_type=Node_ID3) -> None:
        self.data = pd.read_csv(data_path)
        self.root = None
        self.node_type = node_type


    def run(self):
        self.root = self.node_type(self.data)
        print(self.root)

    def predict(self, data_sample):
        curr_node = self.root
        while not curr_node.is_leaf:
            curr_node = curr_node.children[data_sample[curr_node.feature]]

        return curr_node.classification

In [6]:
id3 = DecisionTree('data.csv').run()
c4_5 = DecisionTree('data.csv', node_type = NodeC4_5).run()
cart = DecisionTree('data.csv', node_type = Node_CART).run()

: Split by Outlook
	Overcast: Yes
	Rain: Split by Wind
		Strong: No
		Weak: Yes
	Sunny: Split by Humidity
		High: No
		Low: Yes
		Medium: No

: Split by Outlook
	Overcast: Yes
	Rain: Split by Wind
		Strong: No
		Weak: Yes
	Sunny: Split by Humidity
		High: No
		Low: Yes
		Medium: No

: Split by Outlook
	Overcast: Yes
	Rain: Split by Wind
		Strong: No
		Weak: Yes
	Sunny: Split by Humidity
		High: No
		Low: Yes
		Medium: No

